## Run the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius.

In [1]:
import os
import shutil
import numpy as np
from pathlib import Path
import subprocess
import time

#### Path to STEMMUS_SCOPE

STEMMUS_SCOPE source codes are located in the **private** repository on GitHub https://github.com/EcoExtreML/STEMMUS_SCOPE. You need to clone the repository locally and specify the path to it in the cell below. Make sure you have right access to the repository and you created a local branch for development purposes. 

In [2]:
# path to model repository
path_to_model = Path("path_to_STEMMUS_SCOPE_repository")
path_to_model = Path("/home/alidoost/STEMMUS_SCOPE")

#### set git branch and see status

In [3]:
dev_branch = "name_of_your_branch"
!git -C {path_to_model} checkout {dev_branch}
!git -C {path_to_model} status

Already on 'name_of_your_branch'
On branch name_of_your_branch
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	exe/mccExcludedFiles.log
	exe/readme.txt
	exe/requiredMCRProducts.txt
	exe/run_STEMMUS_SCOPE.sh
	exe/unresolvedSymbols.txt
	java.log.42488
	slurm/
	slurm_901092.out
	slurm_901245.out
	slurm_901391.out
	slurm_901417.out
	slurm_901577.out
	slurm_902191.out
	slurm_908852.out
	test_run_stemmus_scope_snellius.sh
	utils/csv_to_nc/__pycache__/
	utils/get_info_stemmus_scope_snellius.sh

nothing added to commit but untracked files present (use "git add" to track)


#### Update/set config files

In [4]:
# the user must provide the correct path
# path to config file
config_file_path = path_to_model / "config_file_snellius.txt"
# create an empty dict
config = {}
with open(config_file_path, "r") as f:
    for line in f:
        (key, val) = line.split("=")
        config[key] = val.rstrip('\n')

In [5]:
config

{'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'InputPath': '/scratch-shared/ecoextreml/stemmus_scope/input/',
 'OutputPath': '/scratch-shared/ecoextreml/stemmus_scope/output/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'FI-Hyy_1996-2014_FLUXNET2015_Met.nc',
 'VegetationPropertyPath': '/projects/0/einf2480/model_parameters/vegetation_property/',
 'DurationSize': '17520'}

In [6]:
# edit config
config["DurationSize"] = "5"
config

{'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'InputPath': '/scratch-shared/ecoextreml/stemmus_scope/input/',
 'OutputPath': '/scratch-shared/ecoextreml/stemmus_scope/output/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'FI-Hyy_1996-2014_FLUXNET2015_Met.nc',
 'VegetationPropertyPath': '/projects/0/einf2480/model_parameters/vegetation_property/',
 'DurationSize': '5'}

#### Create input directories, prepare input files 

In [7]:
def input_dir(ncfile):
    """Create input directory and prepare input files
    """
    # get start time with the format Y-M-D-HM
    timestamp = time.strftime('%Y%m%d_%H%M')
    station_name = ncfile.split('_')[0]
    # create input directory
    work_dir = Path(config["InputPath"], station_name + '_' + timestamp)
    Path(work_dir).mkdir(parents=True, exist_ok=True)
    print(f"Prepare work directory {work_dir} for the station: {station_name}")
    # copy model parameters to work directory
    shutil.copytree(config["VegetationPropertyPath"], work_dir, dirs_exist_ok=True)
    # update config file for ForcingFileName and InputPath
    config_file_path = Path(work_dir, f"{station_name}_{timestamp}_config.txt")
    with open(config_file_path, 'w') as f:
        for i in config.keys():
            if i == "ForcingFileName":
                f.write(i + "=" + ncfile + "\n")
            elif i == "InputPath":
                f.write(i + "=" + str(work_dir) + "/" + "\n")
            else:
                f.write(i + "=" + config[i] + "\n")

    return work_dir, config_file_path
    

In [8]:
# specify the forcing filenames
# forcing_filenames_list is by default none, if full_run is true, then all stations with forcing
# listed in the ForcingPath will be used.
forcing_filenames_list = ["NL-Hor_2008-2011_FLUXNET2015_Met.nc",
 "ZA-Kru_2000-2002_FLUXNET2015_Met.nc"]

full_run = False
if full_run:
    forcing_filenames_list = Path(config["ForcingPath"]).iterdir()

config_path_dict = {}
work_dir_dict = {}
for ncfile in forcing_filenames_list:
    work_dir_dict[ncfile], config_path_dict[ncfile] = input_dir(ncfile)

Prepare work directory /scratch-shared/ecoextreml/stemmus_scope/input/NL-Hor_20220517_1757 for the station: NL-Hor
Prepare work directory /scratch-shared/ecoextreml/stemmus_scope/input/ZA-Kru_20220517_1757 for the station: ZA-Kru


#### Run the model

In [9]:
# generate a text file
path_to_code = path_to_model / "src"

for ncfile in forcing_filenames_list:
    # set matlab log dir to slurm, otherwise java.log files are created in user home dir
    os.environ['MATLAB_LOG_DIR'] = str(work_dir_dict[ncfile])
    
    # set matlab arguments
    path_to_config = f"'{config_path_dict[ncfile]}'"
    command_line = f'matlab -r "STEMMUS_SCOPE_exe({path_to_config});exit;"'
    args = [command_line, "-nodisplay", "-nosplash", "-nodesktop"]
    
    # run the model
    print(f"Run STEMMUS-SCOPE with the forcing: {ncfile}")
    result = subprocess.run(args, cwd = path_to_code, shell=True)
    result.check_returncode()

Run STEMMUS-SCOPE with the forcing: NL-Hor_2008-2011_FLUXNET2015_Met.nc
MATLAB is selecting SOFTWARE OPENGL rendering.
Opening log file:  /scratch-shared/ecoextreml/stemmus_scope/input/NL-Hor_20220517_1757/java.log.43865

                            < M A T L A B (R) >
                  Copyright 1984-2021 The MathWorks, Inc.
             R2021a Update 3 (9.10.0.1684407) 64-bit (glnxa64)
                                May 27, 2021

 
To get started, type doc.
For product information, visit www.mathworks.com.
 
Reading config from /scratch-shared/ecoextreml/stemmus_scope/input/NL-Hor_20220517_1757/NL-Hor_20220517_1757_config.txt

 The calculations start now 
Run STEMMUS-SCOPE with the forcing: ZA-Kru_2000-2002_FLUXNET2015_Met.nc
MATLAB is selecting SOFTWARE OPENGL rendering.
Opening log file:  /scratch-shared/ecoextreml/stemmus_scope/input/ZA-Kru_20220517_1757/java.log.44704

                            < M A T L A B (R) >
                  Copyright 1984-2021 The MathWorks, Inc.
     

#### Create output directories, prepare output files

In [10]:
# convert csv files to nc files
path_to_utils = Path(path_to_model, "utils/csv_to_nc")
for ncfile in forcing_filenames_list:
    path_to_config = config_path_dict[ncfile]
    result = subprocess.run(["python", Path(path_to_utils, "generate_netcdf_files.py"),
         "--config_file", path_to_config, "--variable_file",
         Path(path_to_utils, "Variables_will_be_in_NetCDF_file.csv")])
    result.check_returncode()

/scratch-shared/ecoextreml/stemmus_scope/output/NL-Hor_2022-05-17-1758/ECdata.csv
Reading variable metadata from /home/alidoost/STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv
Creating /scratch-shared/ecoextreml/stemmus_scope/output/NL-Hor_2022-05-17-1758/NL-Hor_2022-05-17-1758_STEMMUS_SCOPE.nc 
Reading data from file: radiation.csv
Reading data from file: fluxes.csv
Reading data from file: surftemp.csv
Reading data from file: Sim_Temp.csv
Reading data from file: Sim_Theta.csv
Reading data from file: aerodyn.csv
Reading data from file: ECdata.csv
Done writing /scratch-shared/ecoextreml/stemmus_scope/output/NL-Hor_2022-05-17-1758/NL-Hor_2022-05-17-1758_STEMMUS_SCOPE.nc
/scratch-shared/ecoextreml/stemmus_scope/output/ZA-Kru_2022-05-17-1758/ECdata.csv
Reading variable metadata from /home/alidoost/STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv
Creating /scratch-shared/ecoextreml/stemmus_scope/output/ZA-Kru_2022-05-17-1758/ZA-Kru_2022-05-17-1758_STEMMU

#### Generate execuable file

In [11]:
os.environ['MATLAB_LOG_DIR'] = "."
command_line = f'mcc -m ./src/STEMMUS_SCOPE_exe.m -a ./src -d ./exe -o STEMMUS_SCOPE -R nodisplay -R singleCompThread'
result = subprocess.run(command_line, cwd = path_to_model, shell=True)
result.check_returncode()

Opening log file:  ./java.log.45777


#### check changes by git

In [12]:
!git -C {path_to_model} status

On branch name_of_your_branch
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   exe/STEMMUS_SCOPE

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	exe/mccExcludedFiles.log
	exe/readme.txt
	exe/requiredMCRProducts.txt
	exe/run_STEMMUS_SCOPE.sh
	exe/unresolvedSymbols.txt
	java.log.42488
	java.log.45777
	slurm/
	slurm_901092.out
	slurm_901245.out
	slurm_901391.out
	slurm_901417.out
	slurm_901577.out
	slurm_902191.out
	slurm_908852.out
	test_run_stemmus_scope_snellius.sh
	utils/csv_to_nc/__pycache__/
	utils/get_info_stemmus_scope_snellius.sh

no changes added to commit (use "git add" and/or "git commit -a")
